<a href="https://colab.research.google.com/github/Michael-Jimenez-C/CieCom2/blob/main/MapaCII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import folium

In [ ]:
m=folium.Map(location=[4.61,-74.082], zoom_start=10)

puntos={"Bogota":[4.61,-74.082],
        "Soacha":[4.583,-74.217],
        "Madrid":[4.733,-74.267],
        "Facatativa":[4.817,-74.367],
        "Fusagasuga":[4.333,-74.35],
        "Funza":[4.717,-74.217],
        "Chia":[4.85,-74.05],
        "Zipaquira":[5.02836,-73.9992],
        "Girardot":[4.3,-74.8],
        "Villeta":[5.017,-74.467],
        "Macheta":[5.07983,-73.6072],
        "La Vega":[4.99946,-74.3399],
        "Tocaima":[4.45903,-74.634],
        "Mosquera":[4.70612,-74.2307],
        "La Mesa":[4.633,-74.467],
        "Cota":[4.817,-74.1],
        "Gachancipa":[4.99228,-73.8712],
        "La calera":[4.717,-73.967],
        "Choachi":[4.52748,-73.9257],
        "Chipaque":[4.44294,-74.0445]}
for i in puntos:
  folium.Marker(
      puntos[i], popup=f"<i>{i}</i>"
  ).add_to(m)
m